In [ ]:
Use AdventureWorks2017;
GO

 --- Example 1: using default input and output name
-- define Python script
DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'
# assign SQL Server dataset to df variable
df = InputDataSet
 
# return df dataset
OutputDataSet = df';
 
-- define T-SQL query
--- because Subtotal is of type money, you have to convert it to the type that Python can handle ie float.
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = '
  SELECT top 20 t.Name AS Territories, CAST(h.Subtotal AS FLOAT) AS Sales
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
      ON h.TerritoryID = t.TerritoryID ';
 
-- run procedure, using Python script and T-SQL query
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript;    
GO



In [ ]:
----Example 2: using user defined input and output name. note that you have to assign value to your input and output variable in 
---- @input_data_1_name  and @output_data_1_name  respectively. You also set column names.Note that in the result set, you can use the actaul 
---- data type set in the table defination. ie SubTotal is MONEY.
DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'
# assign SQL Server dataset to df
df = RawData
 
# return df dataset
SumData = df';
 
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT top 20 t.Name AS Territories, CAST(h.Subtotal AS FLOAT) AS Sales
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
      ON h.TerritoryID = t.TerritoryID;';
 
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript,
  @input_data_1_name = N'RawData', --input variable name 
  @output_data_1_name = N'SumData' --output variable name
  WITH RESULT SETS(
    (Territories NVARCHAR(50), Sales MONEY)); --output column names 
GO

In [ ]:
-- Example 3
----- GROUP BY : call the object’s groupby function which is a member of the Pandas DataFrame class:
DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'
# calculate sales totals for each territory
df1 = InputDataSet

df2 = df1.groupby("Territories").agg({"Sales": "sum"})
df2 = df2.reset_index()
 
# return aggregated dataset
OutputDataSet = df2';
 
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories, CAST(h.Subtotal AS FLOAT) AS Sales
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID;';
 
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript
  WITH RESULT SETS(
    (Territories NVARCHAR(50), TotalSales MONEY));    
GO

In [ ]:
-- Example 4: use Python case statement to add performance column
----- GROUP BY : call the object’s groupby function which is a member of the Pandas DataFrame class:
DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'
# calculate sales totals for each territory
df1 = InputDataSet

df2 = df1.groupby("Territories").agg({"Sales": "sum"})

# add Performance column
df2["Performance"] = "Acceptable"
df2["Performance"][df2["Sales"] < 7000000] = "Poor"
df2["Performance"][df2["Sales"] > 10000000] = "Excellent"
 

df2 = df2.reset_index()
 
# return aggregated dataset
OutputDataSet = df2';
 
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories, CAST(h.Subtotal AS FLOAT) AS Sales
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID;';

	EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript
  WITH RESULT SETS(
    (Territories NVARCHAR(50), TotalSales MONEY,Performance NVARCHAR(15)));   


In [ ]:
-- Example 5: 
----- GROUP BY Territories and Continents and sum by Sales
DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'
# calculate sales totals for each territory
df1 = InputDataSet

df2 = df1.groupby(["Territories","Continent"]).agg({"Sales": "sum"})

# add Performance column
df2["Performance"] = "Acceptable"
df2["Performance"][df2["Sales"] < 7000000] = "Poor"
df2["Performance"][df2["Sales"] > 10000000] = "Excellent"
 

df2 = df2.reset_index()
 
# return aggregated dataset
OutputDataSet = df2';
 
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories,  t.[Group] as Continent,CAST(h.Subtotal AS FLOAT) AS Sales
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID;';
 
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript
  WITH RESULT SETS(
    (Territories NVARCHAR(50),Continents NVARCHAR(50), TotalSales MONEY,Performance NVARCHAR(15)));    
GO

In [ ]:
-- Example 6: 
/*** GROUP BY Territories and Continents and get the sum ,mean, max and min for Sales
It is important to name your columns using .columns function. Without it, it will be difficult to perform the case statement below because you wont 
know the column name to use in the case statement.
***/
DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'
import calendar as cl

df1 = InputDataSet
df2 = df1.groupby(["Territories", "Continent"] ,as_index=False).agg({"Sales": ["mean", "sum" ,"min", "max"]})
df2.columns = ["Territories", "Continent","Average Sales","Total Sales","Min Sales","Max Sales"]
 
 # add Performance column
df2["Performance"] = "Acceptable"
df2["Performance"][df2["Total Sales"] < 7000000] = "Poor"
df2["Performance"][df2["Total Sales"] > 10000000] = "Excellent"

# return df2 data set
OutputDataSet = df2';
 
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories,  t.[Group] as Continent,CAST(h.Subtotal AS FLOAT) AS Sales
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID;';
 
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript
  WITH RESULT SETS((Territories NVARCHAR(100), 
    Continents  NVARCHAR(20), AvgSales MONEY, TotalSales MONEY, MinSales MONEY, MaxSales MONEY, Performance NVARCHAR(20)));     
GO

In [ ]:
-- Example 7: 
/*** GROUP BY Territories and Continents and get the sum ,mean, max and min for Sales. Also get the mean for TaxAmt
It is important to name your columns using .columns function. Without it, it will be difficult to perform the case statement below because you wont 
know the column name to use in the case statement.
***/
DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'
import calendar as cl
 
df1 = InputDataSet
df2 = df1.groupby(["Territories", "Continent"] ,as_index=False).agg({"Sales": ["mean", "sum" ,"min", "max"],"TaxAmt": "mean"})
df2.columns = ["Territories", "Continent","Average Sales","Total Sales","Min Sales","Max Sales", "Average Tax"]

  
 # add Performance column
df2["Performance"] = "Acceptable"
df2["Performance"][df2["Total Sales"] < 7000000] = "Poor"
df2["Performance"][df2["Total Sales"] > 10000000] = "Excellent"
 
# return df2 data set
OutputDataSet = df2';
 
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories,  t.[Group] as Continent,CAST(h.Subtotal AS FLOAT) AS Sales,CAST(h.TaxAmt AS FLOAT) AS TaxAmt
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID;';
 
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript
  WITH RESULT SETS((Territories NVARCHAR(100), 
    Continents  NVARCHAR(20), AvgSales MONEY, TotalSales MONEY, MinSales MONEY, MaxSales MONEY, AvgTax MONEY,Performance NVARCHAR(20)));     
GO

In [ ]:
DROP PROCEDURE IF EXISTS dbo.GetSalesStats;
GO
 
CREATE PROCEDURE dbo.GetSalesStats
AS
  SET NOCOUNT ON;

-- Example 8 : Wrap Example 7 inside Stored Procedure
/*** GROUP BY Territories and Continents and get the sum ,mean, max and min for Sales. Also get the mean for TaxAmt
It is important to name your columns using .columns function. Without it, it will be difficult to perform the case statement below because you wont 
know the column name to use in the case statement.
***/
DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'
import calendar as cl
 
df1 = InputDataSet
df2 = df1.groupby(["Territories", "Continent"] ,as_index=False).agg({"Sales": ["mean", "sum" ,"min", "max"],"TaxAmt": "mean"})
df2.columns = ["Territories", "Continent","Average Sales","Total Sales","Min Sales","Max Sales", "Average Tax"]

  
 # add Performance column
df2["Performance"] = "Acceptable"
df2["Performance"][df2["Total Sales"] < 7000000] = "Poor"
df2["Performance"][df2["Total Sales"] > 10000000] = "Excellent"
 
# return df2 data set
OutputDataSet = df2';
 
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories,  t.[Group] as Continent,CAST(h.Subtotal AS FLOAT) AS Sales,CAST(h.TaxAmt AS FLOAT) AS TaxAmt
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID;';
 
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript
  WITH RESULT SETS((Territories NVARCHAR(100), 
    Continents  NVARCHAR(20), AvgSales MONEY, TotalSales MONEY, MinSales MONEY, MaxSales MONEY, AvgTax MONEY,Performance NVARCHAR(20)));     
GO


--Test Stored Procedure
EXEC dbo.GetSalesStats

In [ ]:
DROP PROCEDURE IF EXISTS dbo.GetSalesStastsProc;
GO
 
CREATE PROCEDURE dbo.GetSalesStastsProc
  (@low INT, @high INT) --add procedure parameters
AS
  SET NOCOUNT ON;
 
 -- Example 9 : Wrap Example 8 inside Stored Procedure with parameter
/*** GROUP BY Territories and Continents and get the sum ,mean, max and min for Sales. Also get the mean for TaxAmt
It is important to name your columns using .columns function. Without it, it will be difficult to perform the case statement below because you wont 
know the column name to use in the case statement.
***/
DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'
 
df1 = InputDataSet
df2 = df1.groupby(["Territories", "Continent"] ,as_index=False).agg({"Sales": ["mean", "sum" ,"min", "max"],"TaxAmt": "mean"})
df2.columns = ["Territories", "Continent","Average Sales","Total Sales","Min Sales","Max Sales", "Average Tax"]

  
# add Performance column
df2["Performance "] = "Acceptable"
df2["Performance "][df2["Total Sales"] < LowSales] = "Poor"
df2["Performance "][df2["Total Sales"] > HighSales] = "Excellent"
 
# return df2 data set
OutputDataSet = df2';
 
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories,  t.[Group] as Continent,CAST(h.Subtotal AS FLOAT) AS Sales,CAST(h.TaxAmt AS FLOAT) AS TaxAmt
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID;';
 
 EXEC sp_execute_external_script
    @language = N'Python',
    @script = @pscript,
    @input_data_1 = @sqlscript,
    @params = N'@LowSales FLOAT, @HighSales FLOAT', --define Python variables
    @LowSales = @low, --assign procedure parameter to Python variable
    @HighSales = @high --assign procedure parameter to Python variable
  WITH RESULT SETS((Territories NVARCHAR(100), 
    Continents  NVARCHAR(20), AvgSales MONEY, TotalSales MONEY, MinSales MONEY, MaxSales MONEY, AvgTax MONEY,Performance NVARCHAR(20)));     
GO





In [ ]:
---Execute Stored Procedure with parameters

EXEC dbo.GetSalesStastsProc  7000000,  10000000;

In [ ]:
---Example 10. Extensive use of datetime strftime()

DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'

import datetime as dt
 
# calculate sales totals for each territory and month
df1 = InputDataSet

df1.columns = ["Territories", "OrderDates","Total Sales"]

# You can always use **print(list(df1.columns.values))** to get the column names

df1["MonthYearName"] = df1["OrderDates"].dt.strftime("%b  %Y")
df1["MonthYearNo"] = df1["OrderDates"].dt.strftime("%m  %Y")
df1["LongMonthName"] = df1["OrderDates"].dt.strftime("%B")
df1["ShortMonthName"] = df1["OrderDates"].dt.strftime("%b")
df2 = df1.groupby(["Territories", "MonthYearName","MonthYearNo","LongMonthName","ShortMonthName"],as_index=False).agg({"Total Sales": ["sum"]})

OutputDataSet = df2
print(df2)'
 
-- define T-SQL query
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories, h.OrderDate as OrderDate,
    CAST(h.Subtotal AS FLOAT) AS Sales
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID
  WHERE YEAR(h.OrderDate) = 2013;';
 
-- run procedure, using Python script and T-SQL query
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript
  WITH RESULT SETS(
   (Territories NVARCHAR(100),MonthYearName CHAR(10), MonthYearNO CHAR(10), LongMonthName CHAR(10), ShortMonthName CHAR(10),
  TotalSales MONEY));     
GO

In [ ]:
--- Example 11: METHOD 1
--- Add TargetSales computed column
--- Return specific columns  
---Return specific rows    
---Remove ShortMonthName column

DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'

import datetime as dt
 
# calculate sales totals for each territory and month
df1 = InputDataSet

df1.columns = ["Territories", "OrderDates","Total Sales"]

# You can always use **print(list(df1.columns.values))** to get the column names

df1["MonthYearName"] = df1["OrderDates"].dt.strftime("%b  %Y")
df1["MonthYearNo"] = df1["OrderDates"].dt.strftime("%m  %Y")
df1["LongMonthName"] = df1["OrderDates"].dt.strftime("%B")
df1["ShortMonthName"] = df1["OrderDates"].dt.strftime("%b")
df2 = df1.groupby(["Territories", "MonthYearName","MonthYearNo","LongMonthName","ShortMonthName"],as_index=False).agg({"Total Sales": ["sum"]})

# After aggregation, always remember to get your column names as shown below
df2.columns = ["Territories", "MonthYearName","MonthYearNo","LongMonthName","ShortMonthName","Total Sales"]


# Add computed column namely TargetSales to the dataframe df2
df2["TargetSales"] = df2["Total Sales"] * 2

# Filter rows. Below would have failed without df2.columns above bcos Total Sales had no header name after aggregation until df2.columns was used.
df2 = df2[(df2["Territories"] == "Australia") & (df2["Total Sales"] > 300000)]

# Drop ShortMonthName ( Method 1)

df2.drop("ShortMonthName", axis=1, inplace=True)
OutputDataSet = df2'

-- define T-SQL query
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories, h.OrderDate as OrderDate,
    CAST(h.Subtotal AS FLOAT) AS Sales
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID
  WHERE YEAR(h.OrderDate) = 2013;';
 
-- run procedure, using Python script and T-SQL query
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript
WITH RESULT SETS(
   (Territories NVARCHAR(100),MonthYearName CHAR(10), MonthYearNO CHAR(10), LongMonthName CHAR(10), TotalSales MONEY , TargetSales MONEY));     
GO     
GO

In [ ]:
--- Example 11: METHOD 2
--- Add TargetSales computed column
--- Return specific columns  
---Return specific rows    
---Remove ShortMonthName column

DECLARE @pscript NVARCHAR(MAX);
SET @pscript = N'

import datetime as dt
 
# calculate sales totals for each territory and month
df1 = InputDataSet

df1.columns = ["Territories", "OrderDates","Total Sales"]

# You can always use **print(list(df1.columns.values))** to get the column names

df1["MonthYearName"] = df1["OrderDates"].dt.strftime("%b  %Y")
df1["MonthYearNo"] = df1["OrderDates"].dt.strftime("%m  %Y")
df1["LongMonthName"] = df1["OrderDates"].dt.strftime("%B")
df1["ShortMonthName"] = df1["OrderDates"].dt.strftime("%b")
df2 = df1.groupby(["Territories", "MonthYearName","MonthYearNo","LongMonthName","ShortMonthName"],as_index=False).agg({"Total Sales": ["sum"]})

# After aggregation, always remember to get your column names as shown below
df2.columns = ["Territories", "MonthYearName","MonthYearNo","LongMonthName","ShortMonthName","Total Sales"]


# Add computed column namely TargetSales to the dataframe df2
df2["TargetSales"] = df2["Total Sales"] * 2

# Filter rows method 2. Below would have failed without df2.columns above bcos Total Sales had no header name after aggregation until df2.columns was used.
# This method uses location method ie loc.
# <DataFrame>.loc[<rows>, <columns>]
# You specify row information as the first argument and column information as the second argument separating the two by a comma. 
# If you specify column information without row information, you need only include a colon for the first argument as in OutputDataSet below.
# If you specify row information without column information, you can include a colon for the last argument: or leave the colon : out entirely.

df2 = df2.loc[(df2["Territories"] == "Australia") & (df2["Total Sales"] > 300000),:]



# Drop ShortMonthName ( Method 2) by simply passing all the required columns to OutputDataSet. Mehod 2 and 3 are both valid
OutputDataSet = df2[["Territories","MonthYearName", "MonthYearNo","LongMonthName","Total Sales","TargetSales" ]]

# Drop ShortMonthName ( Method 3) by simply passing all the required columns to OutputDataSet using loc
OutputDataSet = df2.loc[:,["Territories","MonthYearName", "MonthYearNo","LongMonthName","Total Sales","TargetSales" ]]'






 
-- define T-SQL query
DECLARE @sqlscript NVARCHAR(MAX);
SET @sqlscript = N'
  SELECT t.Name AS Territories, h.OrderDate as OrderDate,
    CAST(h.Subtotal AS FLOAT) AS Sales
  FROM Sales.SalesOrderHeader h INNER JOIN Sales.SalesTerritory t
    ON h.TerritoryID = t.TerritoryID
  WHERE YEAR(h.OrderDate) = 2013;';
 
-- run procedure, using Python script and T-SQL query
EXEC sp_execute_external_script
  @language = N'Python',
  @script = @pscript,
  @input_data_1 = @sqlscript
WITH RESULT SETS(
   (Territories NVARCHAR(100),MonthYearName CHAR(10), MonthYearNO CHAR(10), LongMonthName CHAR(10), TotalSales MONEY , TargetSales MONEY));     
GO     
GO